# 06 — Intro to FastAPI

In this notebook you’ll learn the core ideas of **FastAPI**, one of the most popular modern Python frameworks for building web APIs.

We’ll explain what FastAPI is, why it’s used so much in AI and backend projects, and finish with a simple, fully working example you can run locally in a few lines of code.

## 🧠 What is FastAPI?

FastAPI is a **modern Python framework for building web APIs** — that is, programs that expose endpoints such as `/chat`, `/predict`, or `/healthz` that other software (or a frontend app) can call.

It’s built on top of **Starlette** (for performance) and **Pydantic** (for data validation). It’s known for being **fast**, **typed**, **async**, and **developer-friendly**.

**Typical use cases:**
- Exposing AI models as APIs (LLMs, RAG pipelines, agents...)
- Creating chat endpoints for web or mobile apps
- Serving predictions, summaries, or analytics
- Health checks, metrics, and monitoring endpoints

👉 In short, FastAPI lets you wrap your Python logic (LangChain, CrewAI, etc.) into a simple web service that can be consumed by anything — a frontend, another backend, or even yourself with `curl`.

## ⚡ Why FastAPI?

FastAPI became the go-to choice for Python APIs because:

- ⚡ **Fast:** built on asyncio + Starlette → handles many requests efficiently.
- 🧩 **Typed:** uses Python type hints to auto-generate validation and docs.
- 📘 **Automatic docs:** built-in Swagger UI at `/docs`.
- 💡 **Productive:** minimal boilerplate, clean syntax.
- 🤝 **Compatible:** works perfectly with Pydantic, LangChain, SQLAlchemy, etc.

It’s ideal for **both small prototypes** and **production-grade applications**.

## 🧩 Basic idea

A FastAPI app is simply a Python file that:

1. Creates a `FastAPI()` instance.
2. Defines endpoints with decorators like `@app.get()` or `@app.post()`.
3. Returns JSON responses.

Let’s build a **minimal working API** that uses a real LLM (OpenAI or Groq) through LangChain.

In [ ]:
# main.py — A minimal FastAPI app calling a real LLM

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional
import os
from dotenv import load_dotenv, find_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# ---- Load environment (.env) ----
_ = load_dotenv(find_dotenv())

app = FastAPI(title="FastAPI + LLM Demo")

# ---- Pydantic schema for /chat ----
class ChatRequest(BaseModel):
    message: str
    thread_id: Optional[str] = None  # optional future use (memory)

# ---- Lazy model factory (OpenAI by default, optional Groq) ----
_chat_model = None

def get_model():
    global _chat_model
    if _chat_model is not None:
        return _chat_model

    provider = os.getenv("LLM_PROVIDER", "openai").lower()

    if provider == "groq":
        from langchain_groq import ChatGroq
        model_name = os.getenv("GROQ_MODEL", "llama-3.1-70b-versatile")
        _chat_model = ChatGroq(model=model_name)
    else:
        model_name = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
        _chat_model = ChatOpenAI(model=model_name)

    return _chat_model


# ---- Simple health endpoint ----
@app.get("/hello")
def say_hello(name: str = "World"):
    return {"message": f"Hello, {name}!"}


# ---- POST /chat: calls the LLM ----
@app.post("/chat")
async def chat(req: ChatRequest):
    """
    Minimal chat endpoint:
    - Builds a short system instruction.
    - Sends user's message to the LLM.
    - Returns the model's text content.
    """
    model = get_model()
    messages = [
        SystemMessage(content="You are a helpful and concise assistant."),
        HumanMessage(content=req.message),
    ]
    try:
        resp = await model.ainvoke(messages)
        return {"reply": resp.content}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"LLM error: {e}")

## 🧪 Running locally

Start your FastAPI app with **Uvicorn**, the ASGI server that powers it.

In your terminal (from this repo’s root):

```bash
uv run uvicorn main:app --reload --port 8000
```

or, if you don’t use `uv`:

```bash
uvicorn main:app --reload --port 8000
```

**Then open your browser at:**

- 👉 http://localhost:8000/docs — interactive Swagger UI
- 👉 http://localhost:8000/hello — test GET request
- 👉 POST /chat — test sending a message

## 🧱 Example requests

**Using your browser:**

```
http://localhost:8000/hello?name=Jaime
```

➡️ Response:
```json
{ "message": "Hello, Jaime!" }
```

**Using curl:**

```bash
curl -X POST http://localhost:8000/chat \
  -H 'Content-Type: application/json' \
  -d '{"message": "Hi there!"}'
```

➡️ Response:
```json
{ "reply": "Hello! How can I assist you today?" }
```

## 🧩 Adding Pydantic models (for clarity)

In real apps, you’ll use **Pydantic models** to define your input and output data. This gives automatic validation and documentation.

```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI(title="Chat API")

class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    reply: str

@app.post("/chat", response_model=ChatResponse)
def chat(req: ChatRequest):
    reply = f"Echo: {req.message}"
    return ChatResponse(reply=reply)
```

Now `/docs` will automatically show request and response schemas.

## 🧩 How this connects with AI apps

This FastAPI layer acts as the **gateway** to your AI logic. For example:

- `@app.post('/chat')` → calls your LangGraph or CrewAI agent.
- `@app.post('/rag')` → triggers a retrieval pipeline.
- `@app.get('/healthz')` → exposes a monitoring endpoint.

Keep this layer **small**, **stateless**, and **easy to test** — the heavy logic lives in your LangChain or LangGraph modules.

## ✅ Summary

- **FastAPI** is a high-performance web framework for APIs.
- Each endpoint is a simple Python function with `@app.get()` or `@app.post()`.
- Requests and responses are automatically parsed and validated.
- Built-in docs make exploration effortless.
- Perfect for connecting LangChain / LangGraph / CrewAI logic into real-world applications.

🚀 Next, you’ll connect this backend to **Streamlit** to build a complete interactive AI app.